<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br><br><br><br><br>
<u><b><font size = 6>A2: Team Project OJ </f></b></u><br><br>
<b><font size = 5> Data Optimization - DAT-5304 </f></b>
<br><br>
<b>Authors</b>          : Carolina Novello Moreira, Jack Daoud & Max Lembke <br>
<b>Date Created</b> : 03/24/2021<br>
<br><br><br><br><br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br>


<b><font size = 5> Instructions </f></b>

This option involves using one of the two real-world datasets that I posted on Canvas. These datasets have information on Unit Sales, Prices, and Gross Margins for several SKU’s (Stock Keeping Units). The OJ Data is from Dominik’s Grocery Store in Chicago. The Milk Dataset is from Peru. Both have weekly data for multiple years. Using this data, you should advise management as to what they should do. Your advice could include marketing strategies (feature and display in OJ Dataset or Loyalty Card usage in Milk Dataset). Some suggestions or ideas are to: 

1.	Estimate Quantity Demanded as a function of price (and similar prices) using a Log Transformation. You can also include marketing strategies in your regression.
2.	Combine Gross Margin information to advise the firm on maximizing profits. To avoid sales cannibalization, simultaneously include multiple products. Depending on how you set this up, you might be able to use Solver in Excel to fin the optimal strategy. It is a NLP.
3.	The estimates of margins and demand are only estimates. Perform Monte Carlo analysis to simulate different strategies even if your estimate is not perfect (draw the regression coefficients from a normal with mean = estimate and standard deviation = standard error). Report on the results from these simulations.

# Set-up 

In [1]:
# import packages 
import pandas as pd #pandas 
import numpy  as np #numpy 
from numpy import arange # for ranges with floats
import math # to be able to use exp()
from pulp import * #pulp everything 
import statsmodels.formula.api as smf # predictive modeling with nice outputs
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from regressors import stats

# set options 
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 50)

# importing file 
df = pd.read_excel('OJ_Data.xlsx')

In [2]:
# helper functions 

# make binary function 
def make_binary(df):
    
    for col in df.columns:
        
        if 'feat' in col \
        or 'disp' in col:
            
            for index, value in df.iterrows():
                
                if df.loc[index, col] >= 0.5:
                    df.loc[index, col] = 1
                else:
                    df.loc[index, col] = 0
        
            df[col] = df[col].astype(int)

# rev and profit function 
def get_rev_and_profit(df):

    # create Revenue columns for each SKU
    for i in range(1,6):
        df['revenue' + str(i)] = 0
     
    
    # compute Revenue using price x sales
    for index, value in df.iterrows():
        for i in range(1,6):
            df.loc[index, 'revenue' + str(i)] \
            = df.loc[index, 'price' + str(i)] * df.loc[index, 'sales' + str(i)]
            
            df['revenue'+str(i)] = df['revenue' + str(i)].round(2)
     
    
    # create Profit columns for each SKU
    for i in range(1,6):
        df['profit' + str(i)] = 0
     
    
    # compute Profit using revenue x gross margin
    for index, value in df.iterrows():
        for i in range(1,6):
            df.loc[index, 'profit' + str(i)] = \
            df.loc[index, 'revenue' + str(i)] * df.loc[index, 'grmar' + str(i)]
            
            df['profit' + str(i)] = df['profit' + str(i)].round(2)

In [3]:
# data cleaning 

# calling make binary function 
make_binary(df)

# calc. rev and profits function 
get_rev_and_profit(df)


# log transformations 

# price
df['log_price1'] = np.log10(df['price1'])
df['log_price2'] = np.log10(df['price2'])
df['log_price3'] = np.log10(df['price3'])
df['log_price4'] = np.log10(df['price4'])
df['log_price5'] = np.log10(df['price5'])

# sales
df['log_sales1'] = np.log10(df['sales1'])
df['log_sales2'] = np.log10(df['sales2'])
df['log_sales3'] = np.log10(df['sales3'])
df['log_sales4'] = np.log10(df['sales4'])
df['log_sales5'] = np.log10(df['sales5'])

# Part 1: Regression 

## Product 1: Tropicana Premium 64 oz

In [4]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales1 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp1 +
                                        feat1""",
                                        data = df)


# step 2: fit the model based on the data
results_1 = lm_best.fit()

# step 3: analyze the summary output
print(results_1.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales1   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     59.46
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           3.56e-34
Time:                        11:49:16   Log-Likelihood:                 57.156
No. Observations:                 116   AIC:                            -98.31
Df Residuals:                     108   BIC:                            -76.28
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4345      0.482      2.977      0.0

In [8]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales1 ~ log_price1 +
                                        log_price3 +
                                        feat1""",
                                        data = df)


# step 2: fit the model based on the data
results_1 = lm_best.fit()

# step 3: analyze the summary output
print(results_1.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales1   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     143.2
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           3.51e-38
Time:                        11:50:18   Log-Likelihood:                 56.954
No. Observations:                 116   AIC:                            -105.9
Df Residuals:                     112   BIC:                            -94.89
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3661      0.323      4.224      0.0

## Product 2: Tropicana Premium 96 oz

In [13]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales2 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp2 +
                                        feat2""",
                                        data = df)


# step 2: fit the model based on the data
results_2 = lm_best.fit()

# step 3: analyze the summary output
print(results_2.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales2   R-squared:                       0.558
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     19.51
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           1.15e-16
Time:                        11:52:24   Log-Likelihood:                 108.06
No. Observations:                 116   AIC:                            -200.1
Df Residuals:                     108   BIC:                            -178.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2913      0.317      7.225      0.0

In [12]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales2 ~ log_price1 +
                                        log_price2 +
                                        feat2""",
                                        data = df)


# step 2: fit the model based on the data
results_2 = lm_best.fit()

# step 3: analyze the summary output
print(results_2.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales2   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     45.79
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           2.17e-19
Time:                        11:51:46   Log-Likelihood:                 107.09
No. Observations:                 116   AIC:                            -206.2
Df Residuals:                     112   BIC:                            -195.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3198      0.257      9.011      0.0

## Product 3: Topicana 64 oz

In [14]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales3 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp3 +
                                        feat3""",
                                        data = df)


# step 2: fit the model based on the data
results_3 = lm_best.fit()

# step 3: analyze the summary output
print(results_3.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales3   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     38.20
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           1.85e-26
Time:                        11:53:24   Log-Likelihood:                -30.148
No. Observations:                 116   AIC:                             76.30
Df Residuals:                     108   BIC:                             98.32
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3313      1.077      2.164      0.0

In [17]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales3 ~ log_price1 +
                                        log_price3 +
                                        log_price4 +
                                        feat3""",
                                        data = df)


# step 2: fit the model based on the data
results_3 = lm_best.fit()

# step 3: analyze the summary output
print(results_3.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales3   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.690
Method:                 Least Squares   F-statistic:                     65.13
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           3.04e-28
Time:                        11:56:00   Log-Likelihood:                -32.340
No. Observations:                 116   AIC:                             74.68
Df Residuals:                     111   BIC:                             88.45
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8873      0.795      1.116      0.2

## Product 4: Minute Maid 64 oz

In [18]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales4 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp4 +
                                        feat4""",
                                        data = df)


# step 2: fit the model based on the data
results_4 = lm_best.fit()

# step 3: analyze the summary output
print(results_4.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales4   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     46.70
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           7.43e-30
Time:                        11:56:37   Log-Likelihood:                 22.112
No. Observations:                 116   AIC:                            -28.22
Df Residuals:                     108   BIC:                            -6.194
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.7515      0.709      3.879      0.0

In [19]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best = smf.ols(formula =  """log_sales4 ~ log_price1 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        feat4""",
                                        data = df)


# step 2: fit the model based on the data
results_4 = lm_best.fit()

# step 3: analyze the summary output
print(results_4.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales4   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     64.56
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           3.93e-31
Time:                        11:58:09   Log-Likelihood:                 20.766
No. Observations:                 116   AIC:                            -29.53
Df Residuals:                     110   BIC:                            -13.01
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0994      0.560      3.748      0.0

## Product 5: Dominick's 64 oz

In [20]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales5 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp5 +
                                        feat5""",
                                        data = df)


# step 2: fit the model based on the data
results_5 = lm_best.fit()

# step 3: analyze the summary output
print(results_5.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales5   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     30.94
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           4.05e-23
Time:                        12:04:48   Log-Likelihood:                -21.400
No. Observations:                 116   AIC:                             58.80
Df Residuals:                     108   BIC:                             80.83
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2585      0.941     -0.275      0.7

In [21]:
# Re-running the the regression without variables with p-values above 0.05

# step 1: build a model
lm_best_opt = smf.ols(formula =  """log_sales5 ~ log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp5 +
                                        feat5""",
                                        data = df)


# step 2: fit the model based on the data
results_5_opt = lm_best_opt.fit()

# step 3: analyze the summary output
print(results_5_opt.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales5   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     36.39
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           6.66e-24
Time:                        12:04:53   Log-Likelihood:                -21.446
No. Observations:                 116   AIC:                             56.89
Df Residuals:                     109   BIC:                             76.17
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2921      0.930     -0.314      0.7

## Marketing Strategies


Bla bla bla

# Part 2: Non-linear Programming

# Part 3: Monte Carlo

## Product 1 

In [ ]:
# week 1

# note: we used the log values here from DF for week 1 
price1 = -1.242699
price2 = -1.219593
price3 = -1.529718
price4 = -1.330509
price5 = -1.604783
disp1 = 0
feat1 = 0

# set up parameters

# intercept 
i1_mu = results_1.params[0]   # coefficient from regression
i1_sigma = results_1.bse[0]   # standard error from regression

# price 1 
p1_mu = results_1.params[1]   # coefficient from regression
p1_sigma = results_1.bse[1]   # standard error from regression

# price 2 
p2_mu = results_1.params[2]   # coefficient from regression
p2_sigma = results_1.bse[2]   # standard error from regression

# price 3
p3_mu = results_1.params[3]   # coefficient from regression
p3_sigma = results_1.bse[3]   # standard error from regression

# price 4 
p4_mu = results_1.params[4]
p4_sigma = results_1.bse[4]

# price 5 
p5_mu = results_1.params[5]   # coefficient from regression
p5_sigma = results_1.bse[5]   # standard error from regression

# disp 1 
dp1_mu = results_1.params[6]  # coefficient from regression
dp1_sigma = results_1.bse[6]  # standard error from regression

# feat 1
f1_mu = results_1.params[7]   # coefficient from regression
f1_sigma = results_1.bse[7]   # standard error from regression

i1 = results_1.params[0]

# set up Simulation parameters
np.random.seed(42)  # fix the random seed for reproducibility 
number_sims = 100_000

all_stats = [] # Save results from each sim here

# begin Simulation Loop
for i in range(number_sims):
    p1 = np.random.normal(p1_mu, p1_sigma)
    p2 = np.random.normal(p2_mu, p2_sigma)
    p3 = np.random.normal(p3_mu, p3_sigma)
    p4 = np.random.normal(p4_mu, p4_sigma)
    p5 = np.random.normal(p5_mu, p5_sigma)
    dp1 = np.random.normal(dp1_mu, dp1_sigma)
    f1 = np.random.normal(f1_mu, f1_sigma)
    #i1 = np.random.normal(i1_mu, i1_sigma)
    demand = i1 + price1*p1 + price2*p2 + price3*p3 + price4*p4 \
                + price5*p5 + dp1 * disp1 + f1* feat1 
    
    all_stats.append([p1,p2,p3,p4,p5,dp1,f1,i1,demand])
                    
# save results in the results dataframe
results_df = pd.DataFrame.from_records(all_stats, 
                                       columns=['p1','p2','p3','p4','p5',
                                                'dp1','f1','i1','demand'])

# plot results
results_df['demand'].plot(kind='hist', title="demand");

In [ ]:
# set up "known" parameters based on information given about the SKUs:
oz_1 = 64
oz_2 = 96
oz_3 = 64
oz_4 = 64
oz_5 = 64

# set up "known" parameters based historical averages (without log transformation):
grmar1_avg = df['grmar1'].mean()
grmar2_avg = df['grmar2'].mean()
grmar3_avg = df['grmar3'].mean()
grmar4_avg = df['grmar4'].mean()
grmar5_avg = df['grmar5'].mean()

# set up "known" parameters based on log of historical averages:
price1_avg_log = np.log10(df['price1'].mean())
price2_avg_log = np.log10(df['price2'].mean())
price3_avg_log = np.log10(df['price3'].mean())
price4_avg_log = np.log10(df['price4'].mean())
price5_avg_log = np.log10(df['price5'].mean())
















# set up random draw parameters for coefficients - SKU5 

# intercept 
intercept_mu = results_5_opt.params[0] # coefficient from regression
intercept_sigma = results_5_opt.bse[0] # standard error from regression

# price SKU2
B1_mu = results_5.params[1]      # coefficient from regression
B1_sigma = results_5.bse[1]      # standard error from regression

# price SKU3
B2_mu = results_5_opt.params[2]  # coefficient from regression
B2_sigma = results_5.bse[2]      # standard error from regression

# price SKU4
B3_mu = results_5_opt.params[3]  # coefficient from regression
B3_sigma = results_5.bse[3]      # standard error from regression

# price SKU5
B4_mu = results_5_opt.params[4]  # coefficient from regression
B4_sigma = results_5.bse[4]      # standard error from regression

# disp SKU5
B5 = results_5_opt.params[5]     # coefficient from regression
B5_sigma = results_5.bse[5]      # standard error from regression

# feat5 SKU5
B6 = results_5_opt.params[6]     # coefficient from regression
B5_sigma = results_5.bse[6]      # standard error from regression

# set up Simulation parameters
np.random.seed(42)  # fix the random seed for reproducibility 

number_sims = 1_000

# Store price5 stats here
p5_stats = [] 

for P in arange(0.01546875, 0.03890625,0.001):

# Store results from each simulation here
    all_stats = [] 

    # Begin Simulation Loop
    for i in range(number_sims):
        intercept = np.random.normal(intercept_mu, intercept_sigma)
        B1 = np.random.normal(B1_mu, B1_sigma)
        B2 = np.random.normal(B2_mu, B2_sigma)   
        B3 = np.random.normal(B3_mu, B3_sigma) 
        B4 = np.random.normal(B4_mu, B4_sigma)
        B5 = np.random.normal(B5_mu, B5_sigma)
        B6 = np.random.normal(B6_mu, B6_sigma)
        sales_5 = math.exp(intercept + B1*price2_avg_log + B2*price3_avg_log + B3*price4_avg_log + B4*(np.log10(P)) + B5*disp5 + B6*feat5) # number of units sold (sales is given in units)
        revenue_5 = sales_5 * oz_5 * P
        profits_5 = revenue_5 * grmar5_avg
        all_stats.append([B4, sales_5, revenue_5, profits_5])

    # Save results in the results dataframe
    results_df = pd.DataFrame.from_records(all_stats, columns=['price5_coefficient', 'units_sold_5', 'revenue_5','profits_5']) 
    
    # 
    p5_stats.append([P,results_df['revenue_5'].mean(),results_df['profits_5'].mean()])

p5_results_df = pd.DataFrame.from_records(p5_stats, columns=['price_per_oz','revenue','profits'])

# Chaeck results
p5_results_df 

# Create a graph with price/oz on the horizontal axis and profits on the vertical axis.
p5_results_df.plot.scatter(x="price_per_oz", y="profits")

# Questions

- Can we drop a product from our analysis if it's p-value is high with all other products i.e. it's negligible in pricing strategies

- Simulate y-intercept? 

    - 3.755434255431918 with simulated i 
    - 3.751853051891881 without simulated i 

